In [1]:
import tensorflow as tf
import numpy as np

In [9]:
rng = np.random.default_rng(seed=42)

In [10]:
X = tf.constant(rng.uniform(low=-1, high=1, size=(6, 2)))
Y = tf.cast((X[:, 1:] - X[:, :1]) > 0, dtype=tf.float64)
X, Y

(<tf.Tensor: shape=(6, 2), dtype=float64, numpy=
 array([[ 0.5479121 , -0.12224312],
        [ 0.71719584,  0.39473606],
        [-0.8116453 ,  0.9512447 ],
        [ 0.5222794 ,  0.57212861],
        [-0.74377273, -0.09922812],
        [-0.25840395,  0.85352998]])>,
 <tf.Tensor: shape=(6, 1), dtype=float64, numpy=
 array([[0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.]])>)

In [4]:
W = tf.Variable(
    initial_value=tf.zeros(shape=(2, 1), dtype=tf.float64),
    trainable=True,
    validate_shape=True
)
B = tf.Variable(
    initial_value=tf.zeros(shape=(1, 1), dtype=tf.float64),
    trainable=True,
    validate_shape=True
)
W, B

(<tf.Variable 'Variable:0' shape=(2, 1) dtype=float64, numpy=
 array([[0.],
        [0.]])>,
 <tf.Variable 'Variable:0' shape=(1, 1) dtype=float64, numpy=array([[0.]])>)

In [5]:
with tf.GradientTape(persistent=True) as tape:
    Z = X @ W + B
    A = tf.math.sigmoid(Z)
    loss = -(Y * tf.math.log(A) + (1 - Y) * tf.math.log(1 - A))
    cost = tf.math.reduce_mean(loss)
A, loss, cost

(<tf.Tensor: shape=(6, 1), dtype=float64, numpy=
 array([[0.5],
        [0.5],
        [0.5],
        [0.5],
        [0.5],
        [0.5]])>,
 <tf.Tensor: shape=(6, 1), dtype=float64, numpy=
 array([[0.69314718],
        [0.69314718],
        [0.69314718],
        [0.69314718],
        [0.69314718],
        [0.69314718]])>,
 <tf.Tensor: shape=(), dtype=float64, numpy=0.6931471805599453>)

In [6]:
dA, dZ, dW, dB = tape.gradient(target=loss, sources=[A, Z, W, B])
dA, dZ, dW, dB

(<tf.Tensor: shape=(6, 1), dtype=float64, numpy=
 array([[ 2.],
        [ 2.],
        [-2.],
        [-2.],
        [-2.],
        [-2.]])>,
 <tf.Tensor: shape=(6, 1), dtype=float64, numpy=
 array([[ 0.5],
        [ 0.5],
        [-0.5],
        [-0.5],
        [-0.5],
        [-0.5]])>,
 <tf.Tensor: shape=(2, 1), dtype=float64, numpy=
 array([[ 1.27832526],
        [-1.00259111]])>,
 <tf.Tensor: shape=(1, 1), dtype=float64, numpy=array([[-1.]])>)

In [7]:
tape.gradient(target=cost, sources=[A, Z, W, B])

[<tf.Tensor: shape=(6, 1), dtype=float64, numpy=
 array([[ 0.33333333],
        [ 0.33333333],
        [-0.33333333],
        [-0.33333333],
        [-0.33333333],
        [-0.33333333]])>,
 <tf.Tensor: shape=(6, 1), dtype=float64, numpy=
 array([[ 0.08333333],
        [ 0.08333333],
        [-0.08333333],
        [-0.08333333],
        [-0.08333333],
        [-0.08333333]])>,
 <tf.Tensor: shape=(2, 1), dtype=float64, numpy=
 array([[ 0.21305421],
        [-0.16709852]])>,
 <tf.Tensor: shape=(1, 1), dtype=float64, numpy=array([[-0.16666667]])>]

In [8]:
tf.matmul(X, dZ, transpose_a=True) / 6

<tf.Tensor: shape=(2, 1), dtype=float64, numpy=
array([[ 0.21305421],
       [-0.16709852]])>

In [69]:
tf.math.reduce_sum(dZ, axis=0)

<tf.Tensor: shape=(1,), dtype=float64, numpy=array([-0.96421025])>

In [7]:
def get_cost(A, B):
    cost = A - B**2
    return cost

In [8]:
A = tf.ones(shape=(3, 1))
B = tf.Variable(A * 3)
with tf.GradientTape(persistent=True) as tape:
    cost = get_cost(A, B)
grad = tape.gradient(target=cost, sources=[B])
grad

[<tf.Tensor: shape=(3, 1), dtype=float32, numpy=
 array([[-6.],
        [-6.],
        [-6.]], dtype=float32)>]

In [18]:
A = tf.ones(shape=(3, 1))
B = tf.Variable(A * 3)
for i in range(3):
    with tf.GradientTape() as t:
        c = get_cost(A, B)
        grad = t.gradient(target=c, sources=[B])
        B.assign_sub(grad[0])
        print(grad[0], B)

tf.Tensor(
[[-6.]
 [-6.]
 [-6.]], shape=(3, 1), dtype=float32) <tf.Variable 'Variable:0' shape=(3, 1) dtype=float32, numpy=
array([[9.],
       [9.],
       [9.]], dtype=float32)>
tf.Tensor(
[[-18.]
 [-18.]
 [-18.]], shape=(3, 1), dtype=float32) <tf.Variable 'Variable:0' shape=(3, 1) dtype=float32, numpy=
array([[27.],
       [27.],
       [27.]], dtype=float32)>
tf.Tensor(
[[-54.]
 [-54.]
 [-54.]], shape=(3, 1), dtype=float32) <tf.Variable 'Variable:0' shape=(3, 1) dtype=float32, numpy=
array([[81.],
       [81.],
       [81.]], dtype=float32)>


In [19]:
A = tf.ones(shape=(3, 1))
B = tf.Variable(A * 3)
with tf.GradientTape() as t:
    for i in range(3):
        c = get_cost(A, B)
grad = t.gradient(target=c, sources=[B])
B.assign_sub(grad[0])
print(grad[0], B)

tf.Tensor(
[[-6.]
 [-6.]
 [-6.]], shape=(3, 1), dtype=float32) <tf.Variable 'Variable:0' shape=(3, 1) dtype=float32, numpy=
array([[9.],
       [9.],
       [9.]], dtype=float32)>


In [22]:
A = tf.ones(shape=(3, 1))
B = tf.Variable(A * 3)
with tf.GradientTape() as t:
    for i in range(3):
        c = get_cost(A, B)
        grad = t.gradient(target=c, sources=[B])
        B.assign_sub(grad[0])
        print(grad[0], B)
        t.reset()


tf.Tensor(
[[-6.]
 [-6.]
 [-6.]], shape=(3, 1), dtype=float32) <tf.Variable 'Variable:0' shape=(3, 1) dtype=float32, numpy=
array([[9.],
       [9.],
       [9.]], dtype=float32)>


ValueError: Tape is not recording.

In [50]:
from itertools import count

In [68]:
class A:
    _instance_count = count(0)
    def __init__(self):
        self.instance_count = next(self._instance_count)
    def test(self, x):
        y = self.get_y__()
        print(f"{x} {y}")
    def get_y__(self):
        return "class A"

In [83]:
a = A()
a.test("hi")
a.instance_count

hi class A


3

In [70]:
class B(A):
    _instance_count = count(0)
    def __init__(self):
        super().__init__()
    def get_y__(self):
        return "class B"

In [81]:
b = B()
b.test("hi")

hi class B


In [75]:
a.instance_count

1

In [82]:
b.instance_count

4